 # Team AASE takes on DeepView
 The aim of this project is to produce a deep learning model that can take 2D slices of a 3D mesh object, and produce an accurate imitation of the object, sort of like a 3D printer! 

### Packages

In [1]:
%matplotlib notebook


In [2]:
import pandas as pd 
import numpy as np 
import os 
import matplotlib.pyplot as plt
import ast
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from skimage import measure
from scipy.ndimage import zoom
import re

### Preprocessing 
The 2D slices given to us are stored in `.raw` files, our first objective is to store them as 3D arrays and combined all of them into a singular dataframe that can be utilized for analysis. We first convert the `.raw` files into `.npy` files. 

In [3]:
mesh_data = pd.read_csv('compiled_mesh_data.csv')
mesh_data.head(6) 

#filter out empty rows
# List of problematic scans
invalid_scans = ['scan_053.raw', 'scan_043.raw', 'scan_089.raw']

# Filter out the rows with these scans
mesh_data = mesh_data[~mesh_data['file_name'].isin(invalid_scans)]

In [4]:
def convert_to_float_array(entry):
    # Convert bytes to string if needed
    if isinstance(entry, bytes):
        entry = entry.decode("utf-8")  # Decode bytes to string
    
    # Use regex to find all floating point or integer numbers in the string
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", entry)
    
    # Convert found numbers to floats and return as a numpy array
    return np.array([float(num) for num in numbers], dtype=np.float32).reshape(-1, 3)  # Reshape if coordinates are (N, 3)

# Ensure all entries in the 'vertices' column are strings
mesh_data['vertices'] = mesh_data['vertices'].apply(lambda x: x.decode("utf-8") if isinstance(x, bytes) else x)

# Apply the function to the 'vertices' column
mesh_clean = mesh_data['vertices'].apply(convert_to_float_array)

# Verify the output
print(mesh_clean.head())  # Check the first few parsed rows

0    [[335.0, 411.0, 404.0], [335.0, 411.0, 405.0],...
1    [[301.0, 376.0, 383.0], [301.0, 379.0, 386.0],...
2    [[461.0, 349.0, 299.0], [461.0, 349.0, 300.0],...
3    [[385.0, 342.0, 392.0], [385.0, 342.0, 393.0],...
4    [[481.0, 374.0, 424.0], [481.0, 374.0, 425.0],...
Name: vertices, dtype: object


### Creating and Training Model

In [8]:
def create_voxel_grid(vertices, grid_size=256):
    """
    Map vertices to a normalized voxel grid.

    Parameters:
    - vertices: numpy array of shape (N, 3).
    - grid_size: Size of the voxel grid along each axis.

    Returns:
    - A voxel grid of shape (grid_size, grid_size, grid_size).
    """
    voxel_grid = np.zeros((grid_size, grid_size, grid_size), dtype=np.float32)

    if vertices.shape[0] == 0:
        print("Empty vertex array.")
        return voxel_grid

    # Normalize vertices to the grid size
    min_vals = vertices.min(axis=0)
    max_vals = vertices.max(axis=0)

    if np.any(max_vals - min_vals == 0):  # Check for flat ranges
        print("Flat vertex range. Check input data.")
        return voxel_grid

    vertices_normalized = ((vertices - min_vals) / (max_vals - min_vals)) * (grid_size - 1)
    indices = vertices_normalized.astype(int)

    # Populate the grid
    for x, y, z in indices:
        x, y, z = np.clip([x, y, z], 0, grid_size - 1)
        voxel_grid[x, y, z] = 1.0

    return voxel_grid


In [10]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

def visualize_voxel_grid_with_slider(voxel_grid):
    """
    Visualize slices of a voxel grid with an interactive slider.
    
    Parameters:
    - voxel_grid: A 3D numpy array representing the voxel grid.
    """
    fig, ax = plt.subplots()
    plt.subplots_adjust(bottom=0.25)
    
    # Initial slice index
    slice_idx = voxel_grid.shape[2] // 2
    im = ax.imshow(voxel_grid[:, :, slice_idx], cmap='gray')
    ax.set_title(f"Slice {slice_idx}")
    
    # Create the slider
    ax_slider = plt.axes([0.25, 0.1, 0.65, 0.03])  # [left, bottom, width, height]
    slider = Slider(ax_slider, 'Slice', 0, voxel_grid.shape[2] - 1, valinit=slice_idx, valstep=1)
    
    # Update function for slider
    def update(val):
        slice_idx = int(slider.val)
        im.set_data(voxel_grid[:, :, slice_idx])
        ax.set_title(f"Slice {slice_idx}")
        fig.canvas.draw_idle()
    
    slider.on_changed(update)
    plt.show()

# Example usage
voxel_grid = create_voxel_grid(mesh_clean)
visualize_voxel_grid_with_slider(voxel_grid)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
class HighResolutionAutoencoder(nn.Module):
    def __init__(self):
        super(HighResolutionAutoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.Conv3d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.Conv3d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(),
            nn.Conv3d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU()
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose3d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(),
            nn.ConvTranspose3d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.ConvTranspose3d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.ConvTranspose3d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [ ]:
def create_voxel_grid(vertices, grid_size=64):
    """
    Create a voxel grid from vertices.

    Parameters:
    - vertices: numpy array of shape (N, 3), where N is the number of vertices.
    - grid_size: Size of the voxel grid along each axis.

    Returns:
    - A 3D numpy array representing the voxel grid.
    """
    voxel_grid = np.zeros((grid_size, grid_size, grid_size), dtype=np.float32)
    
    # Normalize vertices to fit within the grid
    min_vals = vertices.min(axis=0)
    max_vals = vertices.max(axis=0)
    normalized_vertices = (vertices - min_vals) / (max_vals - min_vals) * (grid_size - 1)
    indices = normalized_vertices.astype(int)

    for x, y, z in indices:
        x, y, z = np.clip([x, y, z], 0, grid_size - 1)
        voxel_grid[x, y, z] = 1
    return voxel_grid

# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for vertices in mesh_clean:  # Loop through each set of vertices in the dataset
        # Create a voxel grid
        input_grid = create_voxel_grid(vertices)  # Shape: (64, 64, 64)

        # Convert to PyTorch tensor
        input_tensor = torch.tensor(input_grid, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)  # Shape: (1, 1, 64, 64, 64)

        # Add noise
        noisy_input = input_tensor + torch.randn_like(input_tensor) * 0.01

        # Forward pass
        optimizer.zero_grad()
        outputs = model(noisy_input)
        loss = criterion(outputs, input_tensor)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Monitor training progress
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss / len(mesh_clean):.4f}")

    # Visualize outputs every few epochs
    if (epoch + 1) % 5 == 0:
        with torch.no_grad():
            sample_output = outputs[0, 0].cpu().numpy()
            plt.imshow(sample_output[32], cmap='gray')  # Visualize a slice
            plt.title(f"Epoch {epoch+1} - Reconstruction")
            plt.show()



In [ ]:
# Get a random sample from the dataset
with torch.no_grad():
    model.eval()
    sample_idx = 49  # Change this to visualize different scans
    input_voxel = train_tensor[sample_idx].unsqueeze(0).to(device)  # Add batch dimension
    reconstructed_voxel = model(input_voxel).squeeze().cpu().numpy()  # Remove batch and channel dims

# Visualize the reconstructed voxel grid along different slices
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(reconstructed_voxel[:, :, 32], cmap='gray')  # Slice along Z-axis
plt.title('Z-axis Slice')
plt.subplot(1, 3, 2)
plt.imshow(reconstructed_voxel[:, 32, :], cmap='gray')  # Slice along Y-axis
plt.title('Y-axis Slice')
plt.subplot(1, 3, 3)
plt.imshow(reconstructed_voxel[32, :, :], cmap='gray')  # Slice along X-axis
plt.title('X-axis Slice')
plt.show()


In [ ]:
def plot_mesh_from_data(data, threshold=None):
    # Check data range and dynamically set threshold if none is provided
    data_min, data_max = np.min(data), np.max(data)
    if threshold is None:
        threshold = data_min + (data_max - data_min) * 0.05  # Start with 5% of max range

    print(f"Using threshold: {threshold}")
    print(f"Data range: min={data_min}, max={data_max}")

    # Extract vertices and faces using marching cubes
    try:
        verts, faces, _, _ = measure.marching_cubes(data, level=threshold)
    except ValueError as e:
        print(f"Error in marching cubes: {e}")
        return

    # Extract the x, y, z coordinates of the vertices
    x, y, z = verts[:, 0], verts[:, 1], verts[:, 2]
    
    # Plot the 3D mesh with enhanced visibility
    fig = go.Figure(data=[go.Mesh3d(
        x=x,
        y=y,
        z=z,
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        color='lightblue',
        opacity=0.7,
        lighting=dict(ambient=0.5, diffuse=0.5, roughness=0.1, specular=0.5)
    )])
    
    # Adjust axis ranges dynamically using `aspectmode='data'`
    fig.update_layout(
        scene=dict(
            xaxis=dict(title="X"),
            yaxis=dict(title="Y"),
            zaxis=dict(title="Z"),
            aspectmode='data'  # Adjust aspect ratio based on data range
        ),
        width=800,
        height=800,
        margin=dict(l=50, r=50, b=100, pad=4),
        showlegend=False
    )

    fig.show()


In [ ]:
import numpy as np
from skimage import measure
import plotly.graph_objects as go

def plot_mesh_from_data(data, threshold=None):
    """
    Plot a 3D mesh from voxel grid data.
    
    Parameters:
    - data: 3D numpy array representing the voxel grid.
    - threshold: Threshold value for the marching cubes algorithm.
    """
    # Check data range and dynamically set threshold if none is provided
    data_min, data_max = np.min(data), np.max(data)
    if threshold is None:
        threshold = data_min + (data_max - data_min) * 0.05  # Default: 5% of range

    print(f"Using threshold: {threshold}")
    print(f"Data range: min={data_min}, max={data_max}")

    # Extract vertices and faces using marching cubes
    try:
        verts, faces, _, _ = measure.marching_cubes(data, level=threshold)
    except ValueError as e:
        print(f"Error in marching cubes: {e}")
        return

    # Extract the x, y, z coordinates of the vertices
    x, y, z = verts[:, 0], verts[:, 1], verts[:, 2]
    
    # Plot the 3D mesh with enhanced visibility
    fig = go.Figure(data=[go.Mesh3d(
        x=x,
        y=y,
        z=z,
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        color='lightblue',
        opacity=0.7,
        lighting=dict(ambient=0.5, diffuse=0.5, roughness=0.1, specular=0.5)
    )])
    
    # Adjust layout for better visualization
    fig.update_layout(
        scene=dict(
            xaxis=dict(title="X", range=[-5, 70]),  # Adjust range based on data
            yaxis=dict(title="Y", range=[-5, 70]),
            zaxis=dict(title="Z", range=[-5, 70]),
            aspectmode='cube'  # Keep aspect ratio equal
        ),
        width=800,
        height=800,
        margin=dict(l=50, r=50, b=100, pad=4),
        showlegend=False
    )

    fig.show()


In [ ]:
# Ensure reconstructed_voxel is a 3D numpy array from the model's output
with torch.no_grad():
    model.eval()
    sample_idx = 0  # Change this to visualize a different scan
    input_voxel = train_tensor[sample_idx].unsqueeze(0).to(device)  # Add batch dimension
    reconstructed_voxel = model(input_voxel).squeeze().cpu().numpy()  # Remove batch and channel dims

# Plot the reconstructed voxel grid in 3D
plot_mesh_from_data(reconstructed_voxel, threshold=0.001)  # Adjust threshold if needed


In [ ]:

def plot_2d_slices(original_scan, reconstructed_data, slice_index=5):
    fig, axes = plt.subplots(3, 2, figsize=(10, 15))

    # Original scan X-axis slice (scaled)
    axes[0, 0].imshow(np.squeeze(original_scan[slice_index, :, :]) / 65535, cmap='gray')
    axes[0, 0].set_title("Original X-axis Slice")
    axes[0, 1].imshow(reconstructed_data[slice_index, :, :], cmap='gray')
    axes[0, 1].set_title("Reconstructed X-axis Slice")
    
    # Original scan Y-axis slice (scaled)
    axes[1, 0].imshow(np.squeeze(original_scan[:, slice_index, :]) / 65535, cmap='gray')
    axes[1, 0].set_title("Original Y-axis Slice")
    axes[1, 1].imshow(reconstructed_data[:, slice_index, :], cmap='gray')
    axes[1, 1].set_title("Reconstructed Y-axis Slice")
    
    # Original scan Z-axis slice (scaled)
    axes[2, 0].imshow(np.squeeze(original_scan[:, :, slice_index]) / 65535, cmap='gray')
    axes[2, 0].set_title("Original Z-axis Slice")
    axes[2, 1].imshow(reconstructed_data[:, :, slice_index], cmap='gray')
    axes[2, 1].set_title("Reconstructed Z-axis Slice")
    
    plt.tight_layout()
    plt.show()

#To sid: set the first raw scan here as scan_001 and run the function below
plot_2d_slices(raw_img, reconstructed_tensor, slice_index=5)
